In [0]:
dbutils.fs.ls("/mnt/varunstorage321/raw")

[FileInfo(path='dbfs:/mnt/varunstorage321/raw/2021-03-21/', name='2021-03-21/', size=0, modificationTime=1737526416000),
 FileInfo(path='dbfs:/mnt/varunstorage321/raw/2021-03-28/', name='2021-03-28/', size=0, modificationTime=1737526555000),
 FileInfo(path='dbfs:/mnt/varunstorage321/raw/2021-04-18/', name='2021-04-18/', size=0, modificationTime=1737526660000)]

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source=dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date=dbutils.widgets.get("p_file_date")

In [0]:
races_df=spark.read.csv(f"/mnt/varunstorage321/raw/{v_file_date}/races.csv", header=True)
display(races_df)

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, concat_ws, lit,col
races_final_df=races_df.select('raceId','year','round','circuitId','name','date','time').withColumnRenamed('raceId','race_id').withColumnRenamed('year','race_year').withColumnRenamed('circuitId','circuit_id').withColumn('race_timestamp',to_timestamp(concat_ws(' ', col('date'), col('time')),'yyyy-MM-dd HH:mm:ss')).withColumn('ingestion_date',current_timestamp()).withColumn('data_source',lit(v_data_source)).withColumn('file_date',lit(v_file_date)).select('race_id','race_year','round','circuit_id','name','race_timestamp','ingestion_date','data_source','file_date')
races_final_df.show()

+-------+---------+-----+----------+--------------------+-------------------+--------------------+-----------+----------+
|race_id|race_year|round|circuit_id|                name|     race_timestamp|      ingestion_date|data_source| file_date|
+-------+---------+-----+----------+--------------------+-------------------+--------------------+-----------+----------+
|      1|     2009|    1|         1|Australian Grand ...|2009-03-29 06:00:00|2025-01-27 07:36:...|    testing|2021-04-18|
|      2|     2009|    2|         2|Malaysian Grand Prix|2009-04-05 09:00:00|2025-01-27 07:36:...|    testing|2021-04-18|
|      3|     2009|    3|        17|  Chinese Grand Prix|2009-04-19 07:00:00|2025-01-27 07:36:...|    testing|2021-04-18|
|      4|     2009|    4|         3|  Bahrain Grand Prix|2009-04-26 12:00:00|2025-01-27 07:36:...|    testing|2021-04-18|
|      5|     2009|    5|         4|  Spanish Grand Prix|2009-05-10 12:00:00|2025-01-27 07:36:...|    testing|2021-04-18|
|      6|     2009|    6

In [0]:
#races_final_df.write.mode("overwrite").partitionBy('race_year').parquet("/mnt/varunstorage321/processed/races")

In [0]:
#races_final_df.write.mode('overwrite').format('parquet').saveAsTable('f1_processed.races')

In [0]:
# for delta
races_final_df.write.mode('overwrite').format('delta').saveAsTable('f1_processed.races')

In [0]:
#spark.read.parquet("/mnt/varunstorage321/processed/races").show()

In [0]:
%sql
Select * from f1_processed.races

In [0]:
%sql
desc table extended f1_processed.races

col_name,data_type,comment
race_id,string,null
race_year,string,null
round,string,null
circuit_id,string,null
name,string,null
race_timestamp,timestamp,null
ingestion_date,timestamp,null
data_source,string,null
file_date,string,null
,,


In [0]:
dbutils.notebook.exit("Success")